In [72]:
import csv
import numpy as np
import pandas as pd
import spotipy
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score

c_id = "8accbfbcd8a34c898dd8c7c7348b9258"
c_se ="12f1332683104b0685f2978adebff529"

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))

**Lab | API wrappers - Create your collection of songs & audio features**

In [5]:
import config

In [6]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

# The "sp" variable has two useful funtions:
# The first usefull function is:
# .search(q='',limit=n)
# .search(q="track:"+song_name+" artist:"+artist_name,limit=5) to restrict to a song name and artist.
# Where the "q" keyword is the query you want to perform on spotify: song_name, artist,...
# while The "limit" keyword will limit the number of returned results.
#
# The second usefull function is:
# .audio_features([URL|URI|ID])
# which returns some 'features of the song', that after cleanup, we can use in order to characterize a song.

#results = sp.search(q="Lose yourself",limit=3,market="GB")
#json_results = json.dumps(results, ensure_ascii=True)
#json_results
#results['tracks']['items'][0]['external_urls']["spotify"]

## Searching a playlist

In [7]:
playlist = sp.user_playlist_tracks("spotify", "1C49yxU1XBkoq5yaVDbJwx",market="DE")

## extracting a song from playlist

In [8]:
print(list(playlist.keys())) # Let's look at items and total:
print("Total number of songs in the playlist: ",playlist["total"]) #  Let's check items:
len(playlist["items"]) # It is limited to 100 tracks, we will have to fix it:

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']
Total number of songs in the playlist:  8274


100

In [9]:
playlist["items"][0]

{'added_at': '2015-08-10T17:59:43Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1274663440'},
  'href': 'https://api.spotify.com/v1/users/1274663440',
  'id': '1274663440',
  'type': 'user',
  'uri': 'spotify:user:1274663440'},
 'is_local': False,
 'primary_color': None,
 'track': {'album': {'album_type': 'compilation',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5gznATMVO85ZcLTkE9ULU7'},
     'href': 'https://api.spotify.com/v1/artists/5gznATMVO85ZcLTkE9ULU7',
     'id': '5gznATMVO85ZcLTkE9ULU7',
     'name': 'Lenny Kravitz',
     'type': 'artist',
     'uri': 'spotify:artist:5gznATMVO85ZcLTkE9ULU7'}],
   'external_urls': {'spotify': 'https://open.spotify.com/album/1cW0de5T5fdedlS4YqvyCv'},
   'href': 'https://api.spotify.com/v1/albums/1cW0de5T5fdedlS4YqvyCv',
   'id': '1cW0de5T5fdedlS4YqvyCv',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b273550699c44dd74a7663c6ebfa',
     'width': 640}

In [10]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id,market="DE")
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [11]:
tracks=get_playlist_tracks("spotify", "1C49yxU1XBkoq5yaVDbJwx")
len(tracks)

8274

## extracting the songs ids and features from playlist

In [39]:
list_of_audio_features=[]

for item in range(0,len(tracks)):
    try:
        print(tracks[item]["track"]["id"])
        list_of_audio_features.append(sp.audio_features(tracks[item]["track"]["id"])[0])
        #list_of_audio_features.append(tracks[i]['track']['artists'][0]['name'])
        #list_of_audio_features.append(tracks[i]['track']['name'])
    except:
        pass
len(list_of_audio_features)

2BdSaGVMxZzEeJYjjrlo6n
34x6hEJgGAOQvmlMql5Ige
4ebcE2SmkG7nplvzFAWRu7
0ct6r3EGTcMLPtrXHDvVjc
1mvZErZBp7WZT3HfGBykao
1XGmzt0PVuFgQYYnV2It7A
1GakyY5lyvZGZXZYkk3FjS
4LJhJ6DQS7NwE7UKtvcM52
4PQA5I8Qf99He3SBKtl4Tf
2GWpc4r9gvF9YLEWYJPzLA
1r3Cg2BxENu0pDd8Z8IXjE
5aQmsZCESbhasWW5VARdS8
3Qq7zFuBjiQFmQ8wz55CdJ
67awxiNHNyjMXhVgsHuIrs
0Dc7J9VPV4eOInoxUiZrsL
4ua0IepBEISCWwF8dTJvcU
0n0eMsX7WgEtc7dlscbuKU
00i2HU7TEzzftShjRrDSEF
503OTo2dSqe7qk76rgsbep
0CAfXk7DXMnon4gLudAp7J
0weAUscowxeqDtpCgtbpgp
1AWQoqb9bSvzTjaLralEkT
4SyadrABZJIjeND1HPJS31
3XVozq1aeqsJwpXrEZrDJ9
7pYfyrMNPn3wtoCyqcTVoI
1B75hgRqe7A4fwee3g3Wmu
0RvnQ7hMgtXeJn8VdEhKu1
2mKouqwAIdQnMP43zxR89r
1h2xVEoJORqrg71HocgqXd
5ChkMS8OtdzJeqyybCc9R5
3AszgPDZd9q0DpDFt4HFBy
27AHAtAirQapVldIm4c9ZX
7L93GESzq43UkKUt5FsOCq
0UNTBB02C1aB55Fu4H63qd
6m59VvDUi0UQsB2eZ9wVbH
0ARK753YaiJbpLUk7z5yIM
2b8RCiPb2rtG5AQ1FwyUZX
7vfvZp6KZJpxiaW6rI9ieM
1poBO3pO6fkqPYG6n8J5cX
4hrae8atte6cRlSC9a7VCO
0WccCTWCY2Jtlh1ddAUZxT
26yPacO528yccLPMwVminp
6QewNVIDKdSl8Y3ycuHIei
0JXXm3RqxU8

5QD28FqaM3jTfsqWwvRZwv
5rpRzNcJZqKQXk9PIjreB6
5mc6EyF1OIEOhAkD0Gg9Lc
5LIQBa14xlboDhAjw4tFaR
6mcu7D7QuABVwUGDwovOEh
7bHT9osSq1rwT2yaImzqCi
6G3waRixEIXegXfbRTaIqJ
4ZuzA7bJPki9xqArptIjIe
3l2xQwzdNcor10Pz49JqeN
0mtPNQl5vePVwiI9Vykp7L
582JLMTy2WCHPgKFJ4xCq7
6tsMC7YCLcQxr16O77fkoX
4q71x7UmINGOmompqc6IkC
4wkQmYpAaMe41Rc3sYZ7Vz
5CQbQMgWUL0ejMYvGrnGOo
5UqCQaDshqbIk3pkhy4Pjg
07nH4ifBxUB4lZcsf44Brn
2SHnUyZq0zwmvRIl4WY77G
4J7CKHCF3mdL4diUsmW8lq
0R7YVi7w41Dr9jU5vblAok
0aHbHK2fHzhYVdllnkK7wR
0VoFH3RvCozboo29NcTJ9M
76wbrTbbZcb7OdyA8S0p6Y
2TKayhEbjDQzsTqheVCG7H
2J4Alm5zUad4hB5bY2HhtV
6YwLgicpvVuMt1eE2OldwQ
7xYnUQigPoIDAMPVK79NEq
2kQuhkFX7uSVepCD3h29g5
69gRFGOWY9OMpFJgFol1u0
2ndnVdR8si7CW9T8vJRHyW
1NApivWFEmAJHjNzjd5ddY
2rzBvHM9h36Tpdj7Jdajka
7yIGRf80h3Kwi4GfwJUiQl
3nTlI1r2Vq2THZ9fJSgMmP
2SinkCL9TApmmmsHInM2sj
5qIdM6aQsg9DApUsMI2n8x
4gFUGo1yqElV16pdGU1WJC
6OEolOOk9Gh9bqkdpfIOl0
5l3CML2OnzfNs5RfVgbcLt
0FRCItQsKhyIge8dB6v81o
4kLLWz7srcuLKA7Et40PQR
5dNfHmqgr128gMY2tc5CeJ
3aw9iWUQ3VrPQltgwvN9Xu
1dzQoRqT5uc

66H9jPL1TsOrOYVMAwVjCR
6u1yKoh00WaoSpNp46RzKF
0ct0vSOkP0IZnbPGiDltCR
7Fw5i56my24ZBnGS7hFX2n
5jJ69cMDMC0aeWPjZo6VP2
1Vd6LA8zpHj3oFmG0Z0LxN
2AT8iROs4FQueDv2c8q2KE
7tCHpjktA50ihtkLz6bAnn
4yffABDkB5A9flvr9vQFck
4lOTExIbwaF57eYVmNpYr3
1JcGNoiwifg0MdJMVgJQYx
4Sfa7hdVkqlM8UW5LsSY3F
62yJjFtgkhUrXktIoSjgP2
0LWkaEyQRkF0XAms8Bg1fC
75u9uswC5UMdtv4SQ3xSYf
0Gcj2vsg4VBmkArxOEfrEs
0fsEFwLSJTSHSawhe864Cp
1JDIArrcepzWDTAWXdGYmP
1FkMXuvNvdvaIqZ034viGa
2EGsyBw6qL97qwMy1WmqHR
3bnEhDlFFXeCyyYhbRszW7
5uVji14OWpj9HMKLXdhjAv
16e3Ly7ddGqqnFDiWWWwnc
1ChulFMnwxoD74Me8eX2TU
05wIrZSwuaVWhcv5FfqeH0
5Q7TzTJTgpFnaybWvDA4jd
5zwwXMjE6OrqfVpYGnA9RW
3w3y8KPTfNeOKPiqUTakBh
23eUnTrHxVd4RNnramhnRA
5JvD469hg7VQu2qobOAnuy
56dtB7EzO7EneUgYwX8krC
1JfUjTBKY4xWfEHrOnBhPp
566j2mk7SqZjDS4QfPyhyt
664fvVG2C15AoBlbmpig3p
1LomM3L6atrf79ZL7nHDp4
0aBKFfdyOD1Ttvgv0cfjjJ
0PI6q1rpdfJZy0LeAEE6KC
4p4xKk8VVAaw6YJZyuFfFO
4sFY47f8huQ3k3RJUpU8bh
7rGjz05rxd9MaXUAo7F4kb
2GlYuUkFBHjDJ3mO4nlgDA
7F7qthsVnerS4qwcCmB0iO
1VPUpUwtbpoRVXt0txVU0w
3RRRDZig4RN

1qhcUSJyKubL9ULXz9oJVB
5gBw4CEriBCh5wimCY2dUQ
4kjpe0XSo3sWBn9snjgheF
3fsYp2OCMwkANwqHCAWv5H
1oEaUN7PYQRwcUF52U2M9S
51hSi1p7g2KdXg7fJA1Hgp
6cREm4bV7V79Yxdilx9HDo
4Yf5bqU3NK4kNOypcrLYwU
5CXhE5Q9ePbPlWEeMpL3kA
2So9pYqQ0JWvFhasJfI871
6C10TVvkNRz16MS1Iwz65T
6W5xhp3mIPYKqcJ4Zldybw
7n8pNxkb7sBjXFDeSF3coz
6nurJPNeqAM3ZtE2qwadZq
2xbAMvq8sPxkSyNyARbgHh
3LGzKci9PDKbMNWZ4mNB0C
0qt5f5EL92o8Snzopsv0en
2nNSYajMD4ScHbhthVPqrh
2UIRHeHi72VP5WBMAfkPe5
5y3bkh5lgk7yoi3b5MZj7B
3HMssvf6VwOcVyP2v9AHOq
4nrnfY4UEwzMsJ2fiQ8r5r
1GmMrJoxbF13PliZnpeXcy
1iJFfsfRRBD6zoxwL5NfDq
3SVsgASqZaDDTtL1jaDCyh
70YvYr2hGlS01bKRIho1HM
6PgVDY8GTkxF3GmhVGPzoB
6eJlEcRmeyQfTlDQBDyqkW
31GIUmVfnF4rhITHs90Lqb
0lhou8W3tdUpLBwEEVpO4s
2uGgvmRNgCwXpmxqvJnptU
3E5R61s1v3K6Us4QXKXE5r
19VTqucnxJuH6kilNdweTh
4Whbgmn34wt936MLJNfLqu
22SXyL6pUjwIvGWvgGjLov
0bj0hCTYXv02xeSHyRXuYT
16DhvbuyvJob4Q9GHNYu2n
2nWjhmRImt6LScZIPQMBnM
5Xl8wvp7rKr5lEDUxlGDnB
3LpnzPxkMI6XS4JCbhNeek
0sqE6XPi9VAaWl5NxDyMQK
42T2QQv3xgBlpQxaSP7lnK
4QWU4b9viti7aUefIHrccZ
1MqGKtY9L5q

1vxw6aYJls2oq3gW0DujAo
5FMXrphygZ4z3gVDHGWxgl
62AuGbAkt8Ox2IrFFb8GKV
6EuudaFtwibd4tzcWjonb2
7jmHyHMEqm9MJWiMAneF05
6l6bBZT7h0jbYaBLKsCCTW
6dZKWYSx5YBIme4SfpIHJ0
6OHPdG4tYiHRPUHwf68nRU
2G0GextMwZJLkNxcSZ7ZJ3
3ZpTyU4mF3PR9jLpFA3AAC
0zqmyB1prgGGiOS10sjH8A
4HSzQXLeBD9fV6U4RVDHg9
3QzbEvmnuZIONNv82PylZb
1UqhkbzB1kuFwt2iy4h29Q
2eQOMQkStoRUQyq6HGh1iG
10f0hrbCBiQk9STrFG3EU7
38tKqQuqodiUYOsw6HWyeQ
2Iq6HhIquO7JKr0KfTNLzU
5BxVRqpZi6tIhAap1ZjzVD
2a7W4YNYwFuZ3yVclj4AYO
7JeFi9F9gzrkzdnzyMDquk
72nV5T9HAbKc1sLka9NF6x
4hJNA7VCBfNDly2Lz1SKAt
6hmhG1b4LEyNuashVvuIAo
5gFoAVTN9YlM9uJCrFZtgl
0rHVd8h0cFwgLgPFem6S60
1XoGueAqqKk3QGWzzGvH2y
1SZ5EEmCQ03fRAHnFNYW8N
6FE2iI43OZnszFLuLtvvmg
4FTOFgpECL4tlu8DljL0XM
3D7rcEEifDAaTdeUL7tbpv
5YHR8wBR5jXfGtP766NiAT
51hVRQifMP6k0jJUzDbcpz
1RidHM9BC4M8DpfSQ6YLJh
499WZqSWW1kqS0as389Um1
4cYZRebfwTsVNERxTm29Rb
5iefaBcSTiWQuhHO0ahdrB
0QkWubamAKEvvcOTriSKX3
7vCEPLGrLHqBHyRPPsweYY
1mB5iZe1OrN9gYZtLOklI6
5N0lcnJTtKj4wNDvurHige
0aJrYfARfCKzv6gXjhs4SZ
0qdnPX9umHlQBQuv2sYZlo
55A8N3HXzIe

...

0XH8UBdsn2lgfahaBjsW3i
2EFtUP7BmsjHzhQqI6A4rp
3q3y1t3qSf0wOmmcNTf4K4
1mdJr2ztKV7xjWXOETrTrq
0zn2VSypWDE5ZCRT60dwpN
6CsErGW5cD7raTGR3Zs0SV
52UMQ34rFNENl5E75A7uls
4etE2X6t7GmyiuYLIdI8H5
5CG9Ps5ynNjpKJHmwc95pa
0cThXovxrvU68JuzYUXhJT
3jZHJO0QAb8sbv71DXOD9J
3mkmTUcZtQxuKpyM44lGj6
6FT9FZccFD6nE8dMNslz2n
6HhtmXZi4wh1ssdFGsE55G
0OwvrdjaJOypNsNQHyBWE5
3YJx77Xx8JSwEoxqrkQO5c
1LrLR6D1dfnwIETjb8dsKg
39FS8GMCTRPAgHOyaNnUSl
7zvwxa2s4zIX7y49plhrmo
5rrXkymfQdhNBgBZ3BUVbB
2TwM6FmWDMSBDdKYOsWKdy
6Pkj4nv5K53i64cLVgkVyY
6EAlFsajDK1PVgfhdJbS5Y
1uU8KgP0ny5yxRzYuKaz3i
4eGlvsFXmFqdJrVXe9FbgD
0Omt3rVppHIlYFXNL1VWgL
1GY8zOFi8rC39xXnD0tKO8
7rUPIW0pQoshbOtysdlWX2
2iXcvnD3d1gfLBum0cE5Eg
26fSgIa539TCK3KV76chHU
4Q5HmyNvQHlfylzGY7V3Wr
63xdwScd1Ai1GigAwQxE8y
4ZVZBc5xvMyV3WzWktn8i7
5xcw2nLeZN6wfVADCJbNi2
74JdR9aXE6I74oS1BVRsvb
2u0TWxcEOhIwSzyIAwxdIs
5r2cbcTE0HhvRO2avP8GAh
6Zkq3yk1qrNovJr7YwycAA
1pVFchEqnrpIEtuGrB4srT
0jkBzn7J5jc889TYxwvXy6
5lc9L9FeLBwlJPgEbq9uEw
4Sz8zFchpHQVQVqVyGz5Jb
5NPqyQdAbEld8h4lP4GNOj
5omGxKXszbJ

5jSSzGy92H92J8V6XNsp2X
5Leb8e1Nirv6IFHrmvJNv3
7H3nOoaHiPJPKadrHcMiex
1X6GT7mTCpvfsqIGjgrpP3
24ECU8Mk909Cn8vgEKz6Eu
1yj9BcXgsrwPZYHgFbwfkA
7rt7ywY9yekwaYXk2Zt4Q5
5y1jgbDNgTfxoWXv3FhH2Q
4uZRE3FwwtA6NG7sKPXK7f
1tt6mREDPPuCaely4tIRsV
1r6wqKNFYQbookhQtQz0uW
29KD3nuBrkhn8xYVlyj1cP
3pJJkyoc6cLR1CF6vtUese
6R8DTEYkNXV01NPRDJQZhz
30sL6BVhRXnn1UUtPlgqBe
1ntJLMwDAKD8S1FlwkIOyB
1mvVzDrHXM9F1gI9CVg9r2
2vgCM9swUTBS73jS5FmNQE
4KnqUUq5rHneAwi3eyDQzm
2XXGM19E3Zp32Tkr0S4vEA
21RHUPycLKgIoaAyordGrB
7ldnttFcRZRgfLSNmDbSSg
2H881m3JRA8lpuuwaQL6zy
7kP2qEWVdzeNAXwypkUUZw
7G1dFEXwQVM71EEUQbnctz
1Dvyy0iq9aO1UoPpLdEbv2
7InzQLR47Yhmyd9FIQUnUP
4FzPIalyruWIiefI91qe4B
2NDmBXII8x1j9tyzWvhVAF
2j3PFZXmenUxgjjBUIod7F
2wGSgTmgSF3xjRrHkTc25R
1UIKo1VT6F8rTfS8Mjj5ol
43ZvRCc88Cj6gpMEIpNKQq
44nJlW9IbZ6FOhycOGsNw6
4HuocOuSBJ78RlsFYwTyr5
4tgZYdXbG8kZYW3yYWGaT4
10rjhsIi9Sn2sOoud2l0cU
6JsiDFQRx7GHcuf2UroSYB
5Spsu9HaYhflfAinaP8uk5
2Lyxr25z87YEeFoSrngD3B
5ZdrNnYV5VZWds4WXKf8kf
4oS1UEAjl1Fs2nCpOAbp9Q
24efdUt3KGbZ6wnBLp6j2n
0V7Wn8Fry7K

5icOoE6VgqFKohjWWNp0Ac
0K0zmlScXy0cSnSmnQQNiG
6kPDxVwJH5YtKeytA2rA6M
2RSHsoi04658QL5xgQVov3
6FjOEtsWcmuOksloxG0xx5
3zlBFeAPZEqvA05bKUYVeD
1NoE1pEKbjVDmwLvrMsoop
1NE4ga03C5fRXGPcDIo9pA
0YZ3J8xzGwLOg4yEgST1YK
2x0RZdkZcD8QRI53XT4GI5
1tNJrcVe6gwLEiZCtprs1u
5WHTFyqSii0lmT9R21abT8
4NSW0Km5ZG60L8FthUebPJ
7sRdxkgNb3m3Ciai38ANmd
47kEBNIMqnlET0FLBr5uHS
3JdYopXKugBnS7dhi6RA6v
2AAzsdPqRhRMU1Ykmp6gzA
0A5gdlrpAuQqZ2iFgnqBFW
7AkLPOwjju8taQCJCaIUts
7m08259Q5tSlCoHKq0l5AL
54eOmDg8gQNwl0rtr2l3Wo
2ki7ORQj7Jrx9u4JxmJDXG
1qVxslK1epoLJCfZFhu24L
3sGeefsaBqwyXbwaFybBXl
2ym9NQiaMVnmkLfS7szuXp
1sySpMud8bvFU9SVfcrzrc
5vxX2POnp7NBbaCAIVQix7
03CbQgFxnBFoaaVJTfEqMD
6NJ5Q5QujNwwsst5mdSEfF
0dxpg2n2LMtS1TGXXG9x8b
1tjxQYoB2OS2fRZS43ODLQ
0TK2YIli7K1leLovkQiNik
0vrmHPfoBadXVr2n0m1aqZ
6ImEBuxsbuTowuHmg3Z2FO
0YP8kt3sAYTj6eEyesreQH
7I3skNaQdvZSS7zXY2VHId
4R9KztunyNTH13KtAqePLp
4dRqYKhLVujxiBXcq50YzG
4C5iYZMmSYeIUCtX3uShA7
1z1Hg7Vb0AhHDiEmnDE79l
2dmPVqFpmhNuggEB3niC9S
34nascVHOOWUerN4d6Ydqk
7a0VLUciBUdE2RmpEnod9O
7Lz9J7wBYJ6

1O73ZKgl0THUImkHW6dfAL
6FUHEGAHbt4DbUFp5dNwod
1976dfwMTnJ5gElRhZBuWA
2rmwqU7yzTvzkiaRV53DpT
6N1UWEnnka2kl5g39UnoqI
2uwiBpXGA83hymOwxH6iBH
3BprrMTQ3IlFvHW8oFTTQ1
3zPjMYrFSqucb1y1BgmpPp
6PuoZT4kgw5DrUEdnQ6e01
5Ty6Q5kYKnhNRuZCGrZkxA
3V0W5CfNDMiTKg93wuLFxJ
34Tmk8gwlRRegVVZDRxXxk
2mFRYk9KEriZatDFlfaJlZ
0TxR77PJYC8bHUiENFcpig
3j4viUIDSuB4xfYYlp3Lpa
56YZBgLIp3MtErRWvQwslN
1Dp7ASSZVVOPJ85VdLpjOX
6Ql7rNuoP90aIdgwQkPYBP
4uUG5RXrOk84mYEfFvj3cK
0lwCIykH3nebbJevHiJRhf
3gfve6BgoTkZK3uBkqttz8
4JvFNFTjEtZyEzh8w0YzHF
1PStxB6YvO7PEjHGHYBL04
64dLd6rVqDLtkXFYrEUHIU
7GaiT2Cb8y9d9AE4E6XwFI
7neXef29cogkuASB5rTafo
1yALPtLaDOgEuWvgEUFKpc
0eNfURq0r0oNSwFyw1VuVT
1M2eYYojjNSeFbVPnh20zF
7xY4usHueVpDIWU8P9FnkV
23eLnI8GBwNOVQJJmhbLpS
5OAEQKfQzIB3X4K7TtviZf
0NtGSJX4We9o957Bt9RLXn
6ybAXYfBjpsJL1ARBhtmO3
3b1RzDLDt6XMZqbceYYy0t
2LKOHdMsL0K9KwcPRlJK2v
6xi63rgWgcKkod0nXVnMMT
7bdUH0npz854dv3plG8ieT
48wDtXGVFNZHANlDYTKMAW
0Tyi56S51KsufCIcqRHWCN
0xhHR5lWJCxIYGapEskm7C
75OJrNcGzLR28MPGuDJKo9
0ISIEBuq6GwygeAjubdrxb
0kr277qNoo4

8272

In [104]:
df_Audio_features=pd.DataFrame()
df_Audio_features=df_Audio_features[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","id","duration_ms"]]

**Function for added playlist**

In [40]:
#df_Audio_features = df_Audio_features.append(list_of_audio_features, ignore_index=True)

/var/folders/d0/kyjzvmwn3fdgc5m0t8snt0nm0000gn/T/ipykernel_63546/1643065514.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_Audio_features = df_Audio_features.append(list_of_audio_features, ignore_index=True)


**Cleaning the possible duplicates**

In [22]:
df_Audio_features = df_Audio_features.drop_duplicates('id', keep='last')

**We save the DataFrame in a .CSV**

In [110]:
df_Audio_features.to_csv('Song_DataBase.csv', index=False)

In [111]:
df_Audio_features

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,0.669,0.698,-7.971,0.0713,0.024300,0.000000,0.0782,0.600,83.987,2BdSaGVMxZzEeJYjjrlo6n,261707
1,0.544,0.897,-6.056,0.0471,0.241000,0.000000,0.1300,0.647,157.170,34x6hEJgGAOQvmlMql5Ige,215133
2,0.514,0.942,-8.433,0.0502,0.282000,0.000066,0.1700,0.912,159.847,4ebcE2SmkG7nplvzFAWRu7,175267
3,0.527,0.835,-5.298,0.0433,0.016600,0.000000,0.2490,0.654,125.983,0ct6r3EGTcMLPtrXHDvVjc,176658
4,0.767,0.881,-3.988,0.0301,0.001330,0.000139,0.0551,0.961,130.018,1mvZErZBp7WZT3HfGBykao,209800
...,...,...,...,...,...,...,...,...,...,...,...
8267,0.547,0.932,-2.616,0.0431,0.004090,0.000036,0.0391,0.541,105.009,2GtIcy01Jy17wmNyFoB7IT,238254
8268,0.803,0.917,-4.902,0.0381,0.000237,0.526000,0.0791,0.618,124.011,6ab4oRgVgHHAWLRvFhQsJi,389198
8269,0.614,0.689,-8.956,0.0418,0.012600,0.939000,0.1050,0.503,89.990,0vwMdWKwS4mjeKiYvEq5Vc,129333
8270,0.721,0.857,-3.991,0.0538,0.146000,0.000000,0.0923,0.515,122.092,5S9rv87wRG5DwTxXOQw1XC,169541


In [122]:
HMR = pd.read_csv('HMR.csv')

In [123]:
#We start by asking the user song

User_song = input("Please enter a song you would like to start with : ")
User_song = User_song.lower()
User_song = User_song.title()

#Getting the track_id
results = sp.search(q=User_song,limit=3,market="DE")
results["tracks"]["items"][0]["external_urls"]["spotify"]
track_id=track_id=results["tracks"]["items"][0]["id"]

#Getting the audio_features
User_song_features = sp.audio_features(results["tracks"]["items"][0]["id"] )

IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

Please enter a song you would like to start with : kill bill


In [175]:
sp.audio_features(results["tracks"]["items"][0]["id"] )

[{'danceability': 0.644,
  'energy': 0.728,
  'key': 8,
  'loudness': -5.75,
  'mode': 1,
  'speechiness': 0.0351,
  'acousticness': 0.0543,
  'instrumentalness': 0.169,
  'liveness': 0.161,
  'valence': 0.43,
  'tempo': 88.993,
  'type': 'audio_features',
  'id': '3OHfY25tqY28d16oZczHc8',
  'uri': 'spotify:track:3OHfY25tqY28d16oZczHc8',
  'track_href': 'https://api.spotify.com/v1/tracks/3OHfY25tqY28d16oZczHc8',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3OHfY25tqY28d16oZczHc8',
  'duration_ms': 153947,
  'time_signature': 4}]

In [76]:
if User_song in HMR["song_name"].values:
    print("Nice song ... why not trying this song  : ")
    sample_row = HMR.sample()
    row_values = sample_row.iloc[0].to_list()
    word = " FROM "
    print(f"{row_values[0]} {word} {row_values[1]}")
    print("        \U0001F3B8 \U0001F3B8 \U0001F3B8 \U0001F3B8 \U0001F3B8 \U0001F3B8")

else:
    print("Ok Boomer, let´s go dig for some new oldies together : \U0001F3B8 ")


NameError: name 'User_song' is not defined

In [ ]:
def play_song:
    

**Recomandation Funtion**

In [129]:
def recommender(User_song):
    if User_song in list(HMR['song_name']) or fuzz.partial_ratio(song, list(HMR['song_name']))>85:
        top_song=random.choice(HMR['song_name'])
        result_top = sp.search(q=top_song, limit=1, market='DE')
        track_id_top = result_top['tracks']['items'][0]['id']
        return play_song(track_id_top)
    else:
        result = sp.search(q=User_song, limit=1, market='DE')
        track_id = result["tracks"]["items"][0]["id"]
        song_features=sp.audio_features(track_id)
        df_features = pd.DataFrame(song_features)
        df_features_n = df_features.select_dtypes(include=np.number)
        features_scaled = scaler.transform(df_features_n)
        df_features_scaled = pd.DataFrame(features_scaled, columns=df_features_n.columns)
        cluster = kmeans6.predict(df_features_scaled)[0]
        rec_song = list(np.where(playlist["cluster"]==cluster)[0])
        recommendation = random.choice(rec_song)
        id_recommendation = playlist.iloc[recommendation, 0]
        return play_song(id_recommendation)

In [130]:
recommender(User_song)

TypeError: recommender() takes 0 positional arguments but 1 was given

**User_song Scaler**

In [59]:
Song_DB = HMR.drop(['id'], axis=1, inplace=True)

KeyError: "['id'] not found in axis"

In [113]:
results = sp.search(q=User_song ,limit=3,market="GB")
results["tracks"]["items"][0]["id"]
User_song_Audio_features = sp.audio_features(results["tracks"]["items"][0]["id"])

In [114]:
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [100]:
User_song_Audio_features_df =User_song_Audio_features_df[["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]]
#User_song_Audio_features_df = pd.DataFrame(User_song_Audio_features)
#User_song_Audio_features_df = User_song_Audio_features_df.drop(['id'], axis=1)

In [115]:
User_song_Audio_features_df

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.644,0.735,-5.747,0.0391,0.0521,0.144,0.161,0.418,88.98,153947


In [124]:
scaler.transform(User_song_Audio_features_df)

array([[ 0.30387751, -0.20812737,  0.24237008, -0.49183911, -0.33849376,
         0.60180042, -0.23008386, -0.85390113, -1.30212523, -1.35835223]])